In [2]:
#!python -m pip install featuretools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.5/522.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.5/230.5 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.1/930.1 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 83.3 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for PyMeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=90c97ed91cd8881345cf22ef9dc7af1785732ff22eb9f12e2014a6500706dd06
  Stored in directory: /opt/ml/.cache/pip/wheels/b9/49/19/1ec4bcf2cc76

In [11]:
import torch
import pandas as pd
import numpy as np
# pip install transformers
from transformers import BertTokenizer, BertModel
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets

In [12]:
main_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv') # user-item-time
title_df = pd.read_csv('/opt/ml/input/data/train/titles.tsv', sep='\t') # item-title
year_df = pd.read_csv('/opt/ml/input/data/train/years.tsv', sep='\t') # item-year
director_df = pd.read_csv('/opt/ml/input/data/train/directors.tsv', sep='\t') # item-director
genre_df = pd.read_csv('/opt/ml/input/data/train/genres.tsv', sep='\t') # item-genre(name)
writer_df = pd.read_csv('/opt/ml/input/data/train/writers.tsv', sep='\t') # item-writer

### director별 가장 평점을 가장 많이 받은 영화의 평점수(dir_max_cnt)와, 기여했던 모든 영화의 총합 평점수(dir_sum_cnt)

In [146]:
# director 결측 item 감독을 'nm0000000'로 채워주기
no_dir_item_list = list(set(main_df['item']) - set(director_df['item']))
no_dir_item_df = pd.DataFrame([x for x in zip(no_dir_item_list, ['nm0000000']*len(no_dir_item_list))])
no_dir_item_df.columns=director_df.columns # 컬럼명 동일하게
director_df = pd.concat([director_df, no_dir_item_df]) # 기존 director_df 뒤에 감독없는 영화 추가

In [147]:
director_df['director'].nunique()

1341

In [148]:
item_popularity = pd.read_csv('/opt/ml/input/fighting/FE/item/item_interaction_cnt.csv')

In [149]:
item_popularity.head(3) #6807 row

,item,item_cnt
0,2571,19699
1,2959,18437
2,296,18202


In [150]:
director_df.merge(item_popularity, how='left', on='item')

,item,director,item_cnt
0,1237,nm0000005,1383
1,5147,nm0000005,949
2,7327,nm0000005,751
3,2068,nm0000005,502
4,7396,nm0000005,335
...,...,...,...
7204,73681,nm0000000,58
7205,32728,nm0000000,50
7206,32743,nm0000000,48
7207,106491,nm0000000,74


In [151]:
sum(director_df[director_df['director']=='nm0000005'].merge(item_popularity, how='left', on='item')['item_cnt'].values)


6360

In [152]:
director_list = list(director_df['director'].unique())

In [153]:
result = []
for director in director_list:
    max_cnt = max(director_df[director_df['director']==director].merge(item_popularity, how='left', on='item')['item_cnt'].values) # 해당 감독이 가장 많이 받은 interaction의 수
    sum_cnt = sum(director_df[director_df['director']==director].merge(item_popularity, how='left', on='item')['item_cnt'].values) # 해당 감독이 받은 interaction의 누적수
    result.append((director, max_cnt, sum_cnt))
result_df = pd.DataFrame(result)

In [154]:
result_df.columns = ['director','dir_max_cnt','dir_sum_cnt']
result_df

,director,dir_max_cnt,dir_sum_cnt
0,nm0000005,1383,6360
1,nm0000019,905,3290
2,nm0000033,5953,36859
3,nm0000036,761,1835
4,nm0000037,2493,2814
...,...,...,...
1336,nm2648685,247,294
1337,nm2676052,906,1563
1338,nm2879822,251,471
1339,nm9054338,1015,1380


In [ ]:
### 'nm0000000'의 경우는 감독정보가 안주어진 경우이다. 따라서 중간값을 취해준다.

In [163]:
result_df['dir_max_cnt'].describe()

count     1341.000000
mean      1838.779269
std       2888.695760
min         49.000000
25%        248.000000
50%        691.000000
75%       2061.000000
max      19699.000000
Name: dir_max_cnt, dtype: float64

In [165]:
result_df.loc[result_df['director'] == 'nm0000000', 'dir_max_cnt'] = 691
result_df.loc[result_df['director'] == 'nm0000000', 'dir_sum_cnt'] = 691

In [169]:
result_df

,director,dir_max_cnt,dir_sum_cnt
0,nm0000005,1383,6360
1,nm0000019,905,3290
2,nm0000033,5953,36859
3,nm0000036,761,1835
4,nm0000037,2493,2814
...,...,...,...
1336,nm2648685,247,294
1337,nm2676052,906,1563
1338,nm2879822,251,471
1339,nm9054338,1015,1380


In [170]:
result_df.to_csv("director_max_sum_intercnt.csv", index=False)